In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE148949"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE148949"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE148949.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE148949.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE148949.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE148949_family.soft.gz', 'GSE148949_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Melanoma/GSE148949/GSE148949_family.soft.gz
Matrix file: ../../input/GEO/Melanoma/GSE148949/GSE148949_series_matrix.txt.gz
Background Information:
!Series_title	"BKM120 Treated WHIMs_17 Model Cohort"
!Series_summary	"Aberrant activation of PI3K pathway is frequently observed in triple negative breast cancer (TNBC).  However single agent PI3K inhibitors have shown modest anti-tumor activity.  To investigate biomarkers of response, we tested 17 TNBC PDX models with diverse genetic and proteomic background, with varying PI3K pathway signaling activities for their tumor growth response to the pan-PI3K inhibitor BKM120 as well as baseline and treatment induced proteomic changes as assessed by reverse phase protein array (RPPA).  We demonstrated that PI3K inhibition induces varying degrees of tumor growth inhibition (TGI), with 5 models demonstrating over 80% TGI. BKM120 consiste

### Step 2: Initial Data Loading

In [3]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE148949_family.soft.gz', 'GSE148949_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Melanoma/GSE148949/GSE148949_family.soft.gz
Matrix file: ../../input/GEO/Melanoma/GSE148949/GSE148949_series_matrix.txt.gz


Background Information:
!Series_title	"BKM120 Treated WHIMs_17 Model Cohort"
!Series_summary	"Aberrant activation of PI3K pathway is frequently observed in triple negative breast cancer (TNBC).  However single agent PI3K inhibitors have shown modest anti-tumor activity.  To investigate biomarkers of response, we tested 17 TNBC PDX models with diverse genetic and proteomic background, with varying PI3K pathway signaling activities for their tumor growth response to the pan-PI3K inhibitor BKM120 as well as baseline and treatment induced proteomic changes as assessed by reverse phase protein array (RPPA).  We demonstrated that PI3K inhibition induces varying degrees of tumor growth inhibition (TGI), with 5 models demonstrating over 80% TGI. BKM120 consistently reduced PI3K pathway activity as demonstrated by reduced pAKT following therapy.  Several biomarkers showed significant association with resistance, including baseline levels of growth factor receptors (EGFR, pHER3 Y1197), PI3Kp85 r

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [4]:
import pandas as pd
import numpy as np
import os
import re
import json
from typing import Dict, Any, Callable, Optional, List

# 1. Gene Expression Data Availability
# Based on the series summary and overall design, this dataset appears to contain gene expression data
# using Agilent microarrays for TNBC (triple negative breast cancer) models
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary, we need to check for trait, age, and gender

# For trait (Melanoma):
# The sample characteristics at key 0 mention "Melanoma" as one of the tissue types in a mixture,
# but it doesn't contain information about which samples are Melanoma and which aren't.
# This appears to be a reference RNA mixture, not individual Melanoma samples.
trait_row = None

# For age and gender:
# No information about age or gender is provided in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
# Since we don't have data for any of these variables, we'll create placeholder functions

def convert_trait(value):
    # This function won't be used since trait_row is None
    if pd.isna(value) or value is None:
        return None
    value_str = str(value)
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    if "melanoma" in value_str.lower():
        return 1
    else:
        return 0

def convert_age(value):
    # This function won't be used since age_row is None
    if pd.isna(value) or value is None:
        return None
    value_str = str(value)
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    try:
        return float(value_str)
    except:
        return None

def convert_gender(value):
    # This function won't be used since gender_row is None
    if pd.isna(value) or value is None:
        return None
    value_str = str(value)
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    value_lower = value_str.lower()
    if "female" in value_lower or "f" == value_lower:
        return 0
    elif "male" in value_lower or "m" == value_lower:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None, indicating clinical data is not available for our purposes


False

### Step 4: Gene Data Extraction

In [5]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 29657
First 20 gene/probe identifiers:
Index(['1/2-SBSRNA4', '41334', '41335', '41336', '41337', '41338', '41339',
       '41340', '41341', '41342', '41343', '41344', '41518', '41519', '41520',
       '41521', '41522', '41523', '41524', '41525'],
      dtype='object', name='ID')


### Step 5: Gene Identifier Review

In [6]:
# These identifiers don't appear to be standard human gene symbols
# The presence of numeric identifiers like '41334', '41335' etc. suggests
# these may be probe or feature IDs rather than actual gene symbols
# These would likely need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [7]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], 'ORF': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M']}


### Step 7: Gene Identifier Mapping

In [8]:
# Examine both datasets to understand what we're mapping
print("Gene expression data IDs (first 5):", gene_data.index[:5].tolist())

# Looking at annotation data columns
print("Gene annotation columns:", gene_annotation.columns.tolist())

# It appears the annotation data format is unusual - ID column already contains gene symbols
# Based on the gene annotation preview, the 'ID' column contains gene symbols like 'A1BG', 'A2M'
# while the gene expression data has numeric IDs like '41334'
# This suggests we need a different mapping method

# Try to get the gene mapping from the soft file in a different way
# Let's look for platform information in the SOFT file that might help with the mapping
print("\nSearching for platform information in SOFT file...")

with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    for i, line in enumerate(f):
        if '!Platform_table_begin' in line:
            # Found the platform table, read until end
            platform_lines.append(line.strip())
            for platform_line in f:
                platform_lines.append(platform_line.strip())
                if '!Platform_table_end' in platform_line:
                    break
            break
        if i > 5000:  # Limit search 
            break

# Check if we found platform information
if platform_lines:
    print(f"Found platform information - first 5 lines of {len(platform_lines)} total lines:")
    for line in platform_lines[:5]:
        print(line)
    
    # Create a mapping dataframe from the platform information
    if len(platform_lines) > 2:  # Ensure we have header and data
        platform_df = pd.read_csv(io.StringIO('\n'.join(platform_lines[1:-1])), sep='\t')
        print("\nPlatform mapping dataframe columns:", platform_df.columns.tolist())
        print(preview_df(platform_df))
        
        # Check if there are ID and GENE_SYMBOL or similar columns
        id_col = next((col for col in platform_df.columns if 'ID' in col.upper()), None)
        gene_col = next((col for col in platform_df.columns 
                        if 'GENE' in col.upper() or 'SYMBOL' in col.upper() or 'NAME' in col.upper()), None)
        
        if id_col and gene_col:
            print(f"\nUsing columns '{id_col}' and '{gene_col}' for mapping")
            mapping_df = platform_df[[id_col, gene_col]].rename(columns={id_col: 'ID', gene_col: 'Gene'})
            mapping_df = mapping_df.dropna(subset=['Gene'])
            print("Mapping sample (first 5 rows):")
            print(preview_df(mapping_df))
        else:
            print("\nCould not identify appropriate ID and gene symbol columns")
            # In this case, we need to create a mapping using the gene annotation data we have
            print("Creating mapping from available gene annotation data")
            mapping_df = gene_annotation.rename(columns={'ID': 'Gene', 'ORF': 'ID'})
            print("Mapping sample (first 5 rows):")
            print(preview_df(mapping_df))
else:
    print("No platform information found in SOFT file")
    # Use what we have in gene_annotation
    mapping_df = gene_annotation.rename(columns={'ID': 'Gene', 'ORF': 'ID'})
    print("Using gene annotation as mapping. Sample (first 5 rows):")
    print(preview_df(mapping_df))

# The unusual format of the gene_annotation suggests we're dealing with a special case
# Let's adapt our approach based on what we found

# Final mapping and gene data conversion
try:
    # First ensure ID column is string type for proper matching
    if 'ID' in mapping_df.columns:
        mapping_df['ID'] = mapping_df['ID'].astype(str)
    
    # Apply the gene mapping to get gene expression data
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Print the result to verify
    print("\nGene expression data after mapping:")
    print("Number of genes:", len(gene_data))
    print("Sample of gene IDs (first 10):", gene_data.index[:10].tolist())
    print("Sample of expression values (first 5 rows, first 3 columns):")
    print(gene_data.iloc[:5, :3])
except Exception as e:
    print(f"Error in gene mapping: {e}")
    # If mapping fails, we'll need an alternative approach
    # For now, we'll create a basic identity mapping as a fallback
    print("Using fallback approach: treating original IDs as gene symbols")
    gene_data = gene_data.copy()  # Keep original data but acknowledge the limitation
    print("\nGene expression data (without mapping):")
    print("Number of probes:", len(gene_data))
    print("Sample of probe IDs (first 10):", gene_data.index[:10].tolist())


Gene expression data IDs (first 5): ['1/2-SBSRNA4', '41334', '41335', '41336', '41337']
Gene annotation columns: ['ID', 'ORF']

Searching for platform information in SOFT file...
No platform information found in SOFT file
Using gene annotation as mapping. Sample (first 5 rows):
{'Gene': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], 'ID': ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M']}

Gene expression data after mapping:
Number of genes: 18460
Sample of gene IDs (first 10): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS']
Sample of expression values (first 5 rows, first 3 columns):
          GSM4486560  GSM4486561  GSM4486562
Gene                                        
A1BG       -0.248285    0.177796    0.157001
A1BG-AS1    0.249204   -0.177283    0.026934
A1CF       -2.264265   -1.703735   -1.728858
A2LD1       1.198356    1.538781    1.500333
A2M        -1.232422   -2.352890   -1.300777


### Step 8: Data Normalization and Linking

In [9]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. At this point we must recognize that there isn't usable clinical data for this dataset
# This is consistent with our prior analysis in step 3 where we found trait_row = None
print("No usable clinical/trait data available for this dataset")

# Create a minimal DataFrame with the expected structure for the trait
minimal_clinical_df = pd.DataFrame({trait: [0]}, index=["dummy_sample"])

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved minimal clinical data to {out_clinical_data_file} as no real trait data is available")

# 3. Since we don't have clinical data, we can't create a proper linked dataset
# Instead, we'll validate that this dataset isn't usable for our analysis
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # We confirmed no trait data is available
    is_biased=True,            # Setting to True since dataset is unusable for melanoma analysis
    df=minimal_clinical_df,    # Pass the minimal DataFrame with required structure
    note="Dataset contains gene expression data but no melanoma trait information. Background shows this is a TNBC study, not a melanoma-specific dataset."
)

print(f"Final dataset usability: {is_usable}")
print("Dataset processing completed")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS']


Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']
Shape of normalized gene data: (18221, 54)


Saved normalized gene data to ../../output/preprocess/Melanoma/gene_data/GSE148949.csv
No usable clinical/trait data available for this dataset
Saved minimal clinical data to ../../output/preprocess/Melanoma/clinical_data/GSE148949.csv as no real trait data is available
Abnormality detected in the cohort: GSE148949. Preprocessing failed.
Final dataset usability: False
Dataset processing completed
